# <center>Chess Piece Detector With Customised Pretrained Model</center>

# Mount Drive and Change directory TFOD2.x_Setup dir

In [1]:
%cd /content/drive/MyDrive/Virtual Laboratory/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup


/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup


# Installing Dependencies


In [2]:
!pip install -U --pre tensorflow=="2.*"


!pip install tf_slim

In [3]:
!pip install pycocotools

### Get tensorflow/models or cd to parent directory of the repository.

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (3930/3930), done.
remote: Compressing objects: 100% (3036/3036), done.
remote: Total 3930 (delta 1145), reused 1889 (delta 840), pack-reused 0
Receiving objects: 100% (3930/3930), 49.67 MiB | 17.90 MiB/s, done.
Resolving deltas: 100% (1145/1145), done.
Updating files: 100% (3559/3559), done.


### Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [4]:
%%bash
cd models/research/object_detection/packages/tf2/
pip install .

Processing /content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research/object_detection/packages/tf2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1202 sha256=61b734f20d21f0adef2d2f072593eb42c979896934f1ff567939d5f62430d3bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-yjojuzqf/wheels/17/d5/73/3c30a38fed3f1639640b1fa429780c7155faaf08d454b9ef39
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


# Visiting Tensorflwo2 Detection Model Zoo

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


# Classification Model zoo

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_classification_zoo.md

# Imports

In [5]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display


import pathlib

In [6]:
!pwd

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup


In [7]:
%cd models/research/

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research


In [8]:
!pwd

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research


In [9]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Paches

In [10]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model Preparation

## Loader

 you can try with this link

http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
def load_model(model_name):
  #base_url = 'http://download.tensorflow.org/models/object_detection/'
  base_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

### Custom Model Preparation

In [ ]:
def load_model(model_name):

  model_file = model_name
  model_dir = pathlib.Path(model_file)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research


In [ ]:
# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = '/content/models/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Custom Label Map Loading

In [ ]:
PATH_TO_LABELS = 'training/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

To test your images go to test folder and add your images

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/test_images')

PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('object_detection/test_images/image1.jpg'),
 PosixPath('object_detection/test_images/image2.jpg'),
 PosixPath('object_detection/test_images/image3.jpg')]

### Custom Test Image Loading

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('test_img')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

# Detection

Load an object detection model:

for list of available model visit model zoo. I am loading different model not default one

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'

#model_name =  'efficientdet_d6_coco17_tpu-32'

detection_model = load_model(model_name)

### Custom Model Loading

In [ ]:
model_name = "./content/drive/MyDrive/Virtual Laboratory/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/model/research/exported-models/ChessPieceDetector"
detection_model = load_model(model_name)

Check the model's input signature, it expects a batch of 3-color images of type uint8:

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


And returns several outputs:

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

{'detection_classes': tf.float32,
 'detection_boxes': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32}

In [ ]:
detection_model.signatures['serving_default'].output_shapes

{'detection_classes': TensorShape([None, 100]),
 'detection_boxes': TensorShape([None, 100, 4]),
 'detection_scores': TensorShape([None, 100]),
 'num_detections': TensorShape([None])}

# Inferences

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

Run it on each test image and show the results:

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


Output hidden; open in https://colab.research.google.com to view.

### Custom Model Inferencing

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

# Customised Training For Chess Dataset


1.   Get a dataset from Roboflow or other sources
2.   Keep the dataset along with annotation files in model/research folder
3.    Keep labelmap.pbtext and pipeline.config in model/research/training

**xml_to_csv.py** : xml to csv


**generate_tfrecord.py** : csv to tf_records


**labelmap.pbtxt**

## Download Pretrained Model

In [ ]:
%pwd

'/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research'

In [ ]:
%cd models/research/

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research


In [ ]:
%pwd

'/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research'

Get the model download link from Tensorflow Model Zoo

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2023-07-29 21:17:00--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.193.128, 2607:f8b0:400c:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.193.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  41.5MB/s    in 5.1s    

2023-07-29 21:17:05 (45.9 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [ ]:
!tar -xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [ ]:
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 retinaNet50

In [ ]:
!mkdir training

**Move/upload labelmap.pbtxt and pipeline.config files to training folder**

- pipeline.config file will be downloaded along with model download in retinaNet50 folder(renamed)
- labelmap.pbtxt will be downloaded along with dataset

## Generate TFrecods from csv records

In [11]:
!pwd

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research


In [12]:
ls -lhrt ./dataset_Chess_Piece

total 12K
drwx------ 2 root root 4.0K Jul 30 14:42 train/
drwx------ 2 root root 4.0K Jul 30 14:43 valid/
drwx------ 2 root root 4.0K Jul 30 14:55 test/


### Converting train.csv to train.record

In [ ]:
!python generate_tfrecord.py --csv_input=./dataset_Chess_Piece/train/train.csv --image_dir=./dataset_Chess_Piece/train --output_path=train.record

2023-07-30 14:51:12.230339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research/train.record


### Converting test.csv to test.record

In [ ]:
!python generate_tfrecord.py --csv_input=./dataset_Chess_Piece/test/test.csv --image_dir=./dataset_Chess_Piece/test --output_path=test.record

2023-07-30 15:01:45.664386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research/test.record


### Converting valid.csv to valid.record

In [ ]:
!python generate_tfrecord.py --csv_input=./dataset_Chess_Piece/valid/valid.csv --image_dir=./dataset_Chess_Piece/valid --output_path=valid.record

2023-07-30 14:57:51.121405: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research/valid.record


## Training

## Training

**Modifying model/research/training/pipline.config**

Run object_detection/model_main.py file

Move model_main_tf2.py file to research folder

In [13]:
!python ./model_main_tf2.py --model_dir=./training --pipeline_config_path=./training/pipeline.config

2023-07-30 20:37:01.918124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/pytho

## Resume or Stop Training

simply execute the the same preiously executed code o resume or simply interrupt to stop

In [ ]:
!python ./model_main_tf2.py --model_dir=./training --pipeline_config_path=./training/pipeline.config

## Model Evaluation

In [15]:
!pwd

/content/drive/.shortcut-targets-by-id/1ybC2s2pso1TFHXle2VA2FLdjWbIDUtAc/Virtual_Lab_02/iNeuron_DataScience/16_DL_Computer_Vision/09_Object_Detection_Tensorflow_2.x/TFOD2.x_Setup/models/research


```
!python model_main_tf2.py --model_dir=resume_train --pipeline_config_path=resume_train/pipeline.config

```

In [ ]:
!python ./model_main_tf2.py --model_dir=./training --pipeline_config_path=./training/pipeline.config --checkpoint_dir=training

## Exporting Trained MOdel Using Checkpoint Generated

model/research/object_detection/exporter_main.py file is responsible for export so move it to research dir

In [ ]:
### Exporting the Infernece Graph

!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path training/pipeline.config --trained_checkpoint_dir training/ --output_directory exported-models/ChessPieceDetector